# Binance Test Net API Research

In [1]:
import time
import datetime

import pandas as pd
import numpy as np
import json
import asyncio

from binance import ThreadedWebsocketManager, AsyncClient, BinanceSocketManager


In [2]:
api_key='odwF9bVsSsxjZnckgbSu3NfUgGqqJ2sow4OelwjEttIBB08r3Z5umQL0A03lp2Gd'
api_secret='Q3bcPKvbvlVpzv5BQe3lj7EkWdRhevEp24Oi7TENce6xO0FiXUNQKDa47QTyyKcK'
client = await AsyncClient.create(api_key, api_secret, testnet=True, tld='us')

In [3]:
res = await client.get_exchange_info()
print(json.dumps(res, indent=2))

await client.close_connection()

{
  "timezone": "UTC",
  "serverTime": 1704998257664,
  "rateLimits": [
    {
      "rateLimitType": "REQUEST_WEIGHT",
      "interval": "MINUTE",
      "intervalNum": 1,
      "limit": 6000
    },
    {
      "rateLimitType": "ORDERS",
      "interval": "SECOND",
      "intervalNum": 10,
      "limit": 50
    },
    {
      "rateLimitType": "ORDERS",
      "interval": "DAY",
      "intervalNum": 1,
      "limit": 160000
    },
    {
      "rateLimitType": "RAW_REQUESTS",
      "interval": "MINUTE",
      "intervalNum": 5,
      "limit": 61000
    }
  ],
  "exchangeFilters": [],
  "symbols": [
    {
      "symbol": "ETHBTC",
      "status": "TRADING",
      "baseAsset": "ETH",
      "baseAssetPrecision": 8,
      "quoteAsset": "BTC",
      "quotePrecision": 8,
      "quoteAssetPrecision": 8,
      "baseCommissionPrecision": 8,
      "quoteCommissionPrecision": 8,
      "orderTypes": [
        "LIMIT",
        "LIMIT_MAKER",
        "MARKET",
        "STOP_LOSS_LIMIT",
        "TAKE_PRO

In [4]:
bm = BinanceSocketManager(client)
counter = 0
async with bm.multiplex_socket(['btcusdt@trade', 'ethusdt@trade']) as stream:
    while counter < 10:
        msg = await stream.recv()
        print(msg)
        counter += 1

{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1704998259357, 's': 'BTCUSDT', 't': 360365, 'p': '46532.93000000', 'q': '0.00205000', 'b': 1270934, 'a': 1270930, 'T': 1704998259356, 'm': False, 'M': True}}
{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1704998259357, 's': 'BTCUSDT', 't': 360366, 'p': '46551.11000000', 'q': '0.00400000', 'b': 1270934, 'a': 1270933, 'T': 1704998259356, 'm': False, 'M': True}}
{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1704998262028, 's': 'BTCUSDT', 't': 360367, 'p': '46556.22000000', 'q': '0.00999000', 'b': 1270940, 'a': 1270950, 'T': 1704998262027, 'm': True, 'M': True}}
{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1704998262028, 's': 'BTCUSDT', 't': 360368, 'p': '46554.08000000', 'q': '0.00001000', 'b': 1270945, 'a': 1270950, 'T': 1704998262027, 'm': True, 'M': True}}
{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1704998262029, 's': 'BTCUSDT', 't': 360369, 'p': '46556.23000000', 'q': '0.00570000', 'b':

In [5]:
async with bm.depth_socket('BNBBTC') as stream:
    msg = await stream.recv()
    print(msg)

{'e': 'depthUpdate', 'E': 1704998269361, 's': 'BNBBTC', 'U': 385590, 'u': 385591, 'b': [['0.00661700', '0.00000000'], ['0.00661600', '0.84700000']], 'a': []}


In [2]:
import multiprocessing
import sys

def test(help):
    print(help)

if __name__ == '__main__':
    process = multiprocessing.Process(target=test, args=("help",))
    process.start()
    process.join()


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'test' on <module '__main__' (built-in)>


In [7]:
$jupyter kernelspec list

SyntaxError: invalid syntax (3549128276.py, line 1)